# DataFrame Creation
Author : Patricio Ortiz

The aim of this notebook is to structure the data and create dataframes to work with in the GPT Playground notebook

## Libraries

In [1]:
import pandas as pd
import numpy as np

## Data Handle

In [2]:
# Data samples -> Main Table
df_sam = pd.read_csv('../Data/samples_v2.csv', low_memory=False).iloc[:,:-5].drop(columns=[' id_protocol', ' component_continuity', ' lubricant_continuity', ' machine_continuity', ' number_of_refills',' id_component_change_motive', ' id_lubricant_change_refill',' id_lubricant_change_refill_motive', ' lubricant_change_refill_liters',' lubricant_change_refill_hours', ' state', ' suggestion']).replace("\\N", np.nan).dropna()
df_sam.loc[:,['id_sample', ' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client', ' id_anomaly_type',' id_tracking_state', ' id_condition']] = df_sam.loc[:,['id_sample', ' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client', ' id_anomaly_type',' id_tracking_state', ' id_condition']].astype(int)

# Data components -> Mechanic Components
df_com = pd.read_csv('../Data/components.csv')
df_com.loc[:,['id_component', 'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client']] = df_com.loc[:,['id_component', 'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client']].astype(int)

# Datos ensayos -> Chemical components
df_qui = pd.read_csv('../Data/essays.csv')

# Datos resultados -> Values of meditions and limits
# df_res = pd.read_csv('../Data/results.csv', low_memory=False).replace("\\N", np.nan)
# df_res = df_res[~df_res.id_essay.isin([47,48,57])]
# df_res['value'] = df_res['value'].str.rstrip('.')
# df_res['value'] = df_res['value'].str.replace('\.\.', '.', regex=True)
# df_res['value'] = df_res['value'].str.replace('^(\.)+', '', regex=True)
# df_res.dropna(subset=['lsm','lsc','lim','lic'], how='all', inplace=True)
# df_res.to_csv('../Data/results_v2.csv', index=False)
df_res = pd.read_csv('../Data/results_v2.csv', low_memory=False)
df_res.loc[:,['lic','lim','lsm','lsc']] = df_res.loc[:,['lic','lim','lsm','lsc']].astype(float)

# Datos condiciones -> Conditions label 
df_con = pd.read_csv('../Data/conditions.csv')

# Datos tracking -> Tracking state label
df_tra = pd.read_csv('../Data/tracking.csv')

# Datos anomalias -> Anomalies label
df_ano = pd.read_csv('../Data/anomalies.csv')

In [3]:
# Recover labels
dfk = pd.merge(df_sam, 
               df_con,
               left_on=' id_condition',
               right_on='id').drop(columns=['id', ' id_condition']).rename(columns={' name':' condition'})    

dfk = pd.merge(dfk, 
               df_tra,
               left_on=' id_tracking_state',
               right_on='id').drop(columns=['id', ' id_tracking_state']).rename(columns={' name':' tracking_state'})   
 
dfk = pd.merge(dfk, 
               df_ano,
               left_on=' id_anomaly_type',
               right_on='id').drop(columns=['id', ' id_anomaly_type']).rename(columns={' name':' anomaly'})    


In [4]:
list_cases = ['<1', 'NSD', '9.8.05', '0.000.00', '0.0.00', '0.0.0', '0.004.05', '0-00', 
              '0.2.25', '89.88.56', '<0.1', 'N', 'N/', '/NA', '1.2.446', '0.0.2235', '10.0.37',
              '9.69.6', 'nn', 'n', '', '144.580.0', '2.0.326514', '14.20.154111', '9.9.56',
              '4.19.00', '9.9.2' ]

df_aux = df_res[df_res['value'].isin(list_cases)]
ids = df_aux['id_sample'].unique()
df_res = df_res[~df_res['id_sample'].isin(ids)]
df_res.loc[:,'value'] = df_res.loc[:,'value'].astype(float).round(1)

# Get dataframe with values
df_val = pd.merge(df_res, df_qui, 
                  left_on='id_essay',
                  right_on='id').drop(columns=['id','id_essay']).rename(columns={'value':'valor',
                                                                                 'name':'elemento',})

df2 = df_val.copy()

df_val.rename(columns={ 'lsm' :'limite superior marginal',
                        'lsc' :'limite superior condenatorio',
                        'lim' :'limite inferior marginal',
                        'lic' :'limite inferior condenatorio', }, inplace=True)

df_val = df_val[(df_val.valor >= df_val['limite superior marginal']) | (df_val.valor <= df_val['limite inferior marginal'])]
df_val = df_val[df_val.valor > 0]

df_val.tail()

,id_sample,valor,limite inferior condenatorio,limite inferior marginal,limite superior marginal,limite superior condenatorio,elemento
6821302,384807,3.5,NaN,NaN,1.5,3.0,Hollín %
6821394,385155,1.5,NaN,NaN,1.5,3.0,Hollín %
6821423,385292,2.0,NaN,NaN,1.5,3.0,Hollín %
6821451,385368,2.6,NaN,NaN,1.5,3.0,Hollín %
6821453,385374,1.7,NaN,NaN,1.5,3.0,Hollín %


In [5]:
# Obtain data of components
dfi = pd.merge(dfk, df_com,
               left_on=[' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client'],
               right_on=['id_component', 'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client']).drop(columns=['id_component',' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client',
                                                                                                                                      'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client', 'id_component_brand', 'id_component_model',
                                                                                                                                      'id_machine_brand', 'id_machine_model'])

dfi.head() # THIS IS USED AS DF_GENERAL

,id_sample,condition,tracking_state,anomaly,name_component,name_component_brand,name_component_model,name_component_type,name_machine,name_machine_type,name_machine_brand,name_machine_model,name_site,name_client
0,54065,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-1,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
1,57247,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-1,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
2,59059,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-1,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
3,54066,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-2,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
4,57246,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-2,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM


In [6]:
df_val.to_csv('datos procesados/valores.csv', index=False)
dfi.to_csv('datos procesados/componentes.csv', index=False)
df2.to_csv('datos procesados/valores_raw.csv', index=False)